# Why Do We Need LangChain Prompt Templates?

LangChain prompt templates provide a structured and reusable way to define prompts when working with large language models (LLMs). Rather than hardcoding prompts or manually formatting input each time, prompt templates allow developers to:

- **Improve Reusability**: Define prompts once and reuse them across different use cases or workflows by simply updating input variables.

- **Maintain Consistency**: Ensure consistent formatting and phrasing across all prompts, which leads to more reliable and predictable model behavior.

- **Simplify Customization**: Easily inject dynamic variables (e.g., user inputs, styles, or context) into prompts without manual string manipulation.

- **Enhance Readability and Maintainability**: Clearly separate prompt structure from the logic of the application, making the codebase cleaner and easier to understand.

In summary, LangChain prompt templates streamline prompt creation and make it easier to build robust, scalable, and maintainable LLM-powered applications.

# Setting Up Work Environment

In [1]:
!pip install --upgrade google-generativeai

In [2]:
!pip install -q -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.4/206.4 kB 5.6 MB/s eta 0:00:00


In [3]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [4]:
!pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [5]:
import os
from google import genai
import google.generativeai as ggenai
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown

from PIL import Image
from google.genai import types

from IPython.display import HTML

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

In [7]:
# Set up the API key (Replace 'YOUR_API_KEY' with your actual Gemini API key)
key = userdata.get('genai_api')
client = genai.Client(api_key=key)

List the set of available models

In [8]:
ggenai.configure(api_key=key)

models = ggenai.list_models()
for model in models:
    print(model.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
model

Create a helper function to make it easier to use prompts and look at generated outputs.

In [9]:
def get_completion(prompt, model="gemini-1.5-flash", temperature = 0):
  response = client.models.generate_content(
      model=model,
      contents=prompt,
      config=types.GenerateContentConfig(max_output_tokens=200, temperature = temperature)
      )
  return response.text  # Extract the generated text

# Prompt Template

In [10]:
chat = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",       # or another Gemini model of your choice
    temperature=0.0,
    google_api_key=key# exact same temperature as your ChatOpenAI example
)

In [11]:
chat

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x798a81b30110>, default_metadata=(), model_kwargs={})

- **define the template string**

In [12]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

- To enable repeated reuse of this template, we begin by importing LangChain’s `ChatPromptTemplate`.Following the import, we construct a prompt template using the template string defined earlier.

In [13]:
prompt_template = ChatPromptTemplate.from_template(template_string)

In [14]:
prompt_template

ChatPromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'), additional_kwargs={})])

In [15]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

- We can also display the input variables to verify that the template correctly recognizes both `style` and `text` as its input parameters.

In [16]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

# Examples

- Next, we define the desired style, which represents the tone or format to which the customer message should be translated. We will refer to this as the **customer style**. For demonstration purposes, we will use the same customer email as in the previous example.

### Ex 1

In [17]:
customer_style = """American English \
in a calm and respectful tone
"""

In [18]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [19]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


- Finally, we pass the prompt to the language model by invoking the `chat` object, which was previously initialized. By printing the content of the customer response, we can observe the output specifically, the original message translated from English pirate to polite American English.

In [21]:
customer_response = chat.invoke(customer_messages)

customer_response.content

"I'm quite upset that my blender lid came off unexpectedly, resulting in smoothie splattered on my kitchen walls.  To make matters worse, the warranty doesn't appear to cover the cost of cleaning up the mess. I would appreciate your assistance with this matter."

### Ex 2

In [23]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [25]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

service_messages[0].content

"Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n```\n"

In [26]:
service_response = chat.invoke(service_messages)

service_response.content

"Ahoy there, matey!  Regarding yer kitchen appliance warranty, shiver me timbers, but it seems the cleaning o' yer blender be not covered.  It appears, through no fault o' our own, that ye were a bit careless in its operation, forgettin' the lid, aye?  Avast there,  we be sorry for the inconvenience, but such mishaps be not included in the warranty's protection.  Farewell, and fair winds!"